In [1]:
# Import Libraries
import os, sys, findspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F, Window as W

In [2]:
# Uses Java 17 & Python 3.11
os.environ["JAVA_HOME"] = "/opt/homebrew/Cellar/openjdk@17/17.0.17/libexec/openjdk.jdk/Contents/Home"
os.environ["PYSPARK_PYTHON"] = sys.executable
os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable
# Builds PySpark session for 4 local cores with 10GB RAM
findspark.init()
spark = (
    SparkSession.builder
    .appName("SpotifyRec")
    .master("local[4]")
    .config("spark.driver.memory", "10g")
    .config("spark.sql.adaptive.enabled", "true")
    .getOrCreate()
)
# Remove error logs for cleaner output
spark.sparkContext.setLogLevel("ERROR")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/11/06 00:12:18 WARN Utils: Your hostname, Ethans-MacBook-Pro.local, resolves to a loopback address: 127.0.0.1; using 100.64.14.129 instead (on interface en0)
25/11/06 00:12:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/06 00:12:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
# Interactions DataFrame
playlist_tracks = spark.read.parquet("parquet_data/playlist_tracks")
playlist_tracks.createOrReplaceTempView("playlist_tracks")

# User DataFrame
playlists = spark.read.parquet("parquet_data/playlists")
playlists.createOrReplaceTempView("playlists")

# Items DataFrame
tracks = spark.read.parquet("parquet_data/tracks")
tracks.createOrReplaceTempView("tracks")

# Item Features
# Read tracks_features parquet for model training, create a temporary SQL table
track_features = spark.read.parquet('parquet_data/track_features')
track_features.createOrReplaceTempView("track_features")

# User Features
# Read playlist_features parquet for model training, create a temporary SQL table
playlist_features = spark.read.parquet('parquet_data/playlist_features')
playlist_features.createOrReplaceTempView("playlist_features")

# Item-User Interactions
# Read edges parquet for model training, create a temporary SQL table
edges = spark.read.parquet('parquet_data/edges')
edges.createOrReplaceTempView("edges")